In [4]:
pip install openai

  Using cached openai-1.57.4-py3-none-any.whl.metadata (24 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached openai-1.57.4-py3-none-any.whl (390 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key = 'lms'
)

class PreprocessingAgent:
    def __init__(self):
        self.units_regex = r"\d+\s*(miles|kilometers|pounds|kilograms)"

    def identify_critical_segments(self, dialogue):
        # Rule-based unit detection
        critical_entities = []
        if re.search(self.units_regex, dialogue):
            critical_entities.append("Unit Conversion")

        # OpenAI-based detection of expressions
        expressions = self.detect_expressions(dialogue)
        if expressions:
            critical_entities.extend([f"Expression: {expr}" for expr in expressions])

        # OpenAI-based cultural reference detection
        cultural_reference = self.detect_cultural_reference(dialogue)
        if cultural_reference:
            critical_entities.append(f"Cultural Reference: {cultural_reference}")

        if critical_entities:
            return f"Detected Critical Entities: {', '.join(critical_entities)}"

        return "Standard"

    def detect_cultural_reference(self, dialogue):
        # LLM-based detection with OpenAI GPT model
        prompt = f"Analyze the following text and identify if it contains any cultural references or expressions. If so, explain:\n\n\"{dialogue}\""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in detecting cultural references in text."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def detect_expressions(self, dialogue):
        # Use OpenAI GPT model to detect expressions dynamically
        prompt = f"Analyze the following text and list all idiomatic expressions or common phrases:\n\n\"{dialogue}\""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in identifying idiomatic expressions."},
                {"role": "user", "content": prompt}
            ]
        )
        expressions = response.choices[0].message.content.strip()
        return [expr.strip() for expr in expressions.split(',') if expr.strip()]

class TranslationAgent:
    def __init__(self):
        pass

    def translate(self, dialogue):
        # Use OpenAI GPT model for translation
        prompt = f"Translate the following English text into French:\n\n\"{dialogue}\""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert translator from English to French."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

class ContextualAgent:
    def __init__(self):
        pass

    def adapt_context(self, dialogue):
        # Use OpenAI GPT model for contextual adaptation
        prompt = f"Refine the following French translation to better fit the context:\n\n\"{dialogue}\""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in refining translations for better contextual accuracy."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

class ManualValidationAgent:
    def __init__(self):
        pass

    def validate(self, original, contextual):
        print("Original:", original)
        print("Contextual Translation:", contextual)
        user_input = input("Enter your validation or press Enter to accept: ")
        return user_input if user_input.strip() else contextual

class OptimizationAgent:
    def __init__(self, max_chars=42):
        self.max_chars = max_chars

    def optimize(self, dialogue):
        if len(dialogue) > self.max_chars:
            return dialogue[:self.max_chars-3] + "..."
        return dialogue

class DialogueTranslationPipeline:
    def __init__(self):
        self.preprocessing_agent = PreprocessingAgent()
        self.translation_agent = TranslationAgent()
        self.contextual_agent = ContextualAgent()
        self.manual_validation_agent = ManualValidationAgent()
        self.optimization_agent = OptimizationAgent()

    def process_dialogue(self, dialogue):
        # Step 1: Preprocessing
        critical_type = self.preprocessing_agent.identify_critical_segments(dialogue)
        print(f"Critical Segment Type: {critical_type}")

        # Step 2: Translation
        translated = self.translation_agent.translate(dialogue)
        print(f"Translated: {translated}")

        # Step 3: Contextual Adaptation
        contextual = self.contextual_agent.adapt_context(translated)
        print(f"Contextual Translation: {contextual}")

        # Step 4: Manual Validation
        validated = self.manual_validation_agent.validate(dialogue, contextual)

        # Step 5: Optimization
        optimized = self.optimization_agent.optimize(validated)
        print(f"Final Optimized Translation: {optimized}")

        return optimized




In [8]:
# Example usage
pipeline = DialogueTranslationPipeline()
dialogue = "The meeting was a piece of cake, and he's gonna break the ice."
result = pipeline.process_dialogue(dialogue)
print("Result:", result)

Critical Segment Type: Detected Critical Entities: Expression: In the provided text, Expression: there are two idiomatic expressions:

1. **"A piece of cake"**: This idiom means that something is very easy to accomplish.

2. **"Break the ice"**: This expression refers to doing or saying something to relieve tension or get a conversation started in a social setting.

These phrases are commonly used to convey ease and the initiation of interaction, Expression: respectively., Cultural Reference: The text you provided does contain cultural references and expressions that are commonly used in English-speaking contexts.

1. **"The meeting was a piece of cake":** This is an idiomatic expression meaning that something was very easy to accomplish or required little effort. The phrase "piece of cake" originates from the notion that eating cake is enjoyable and simple, thus likening an easy task to this experience. It reflects a cultural tendency in English to use food-related metaphors for ease 